## ДЗ 3 Теория Информации
#### Камаев Виктор БИБ 214


In [152]:
import numpy as np
from collections import defaultdict

У многих функций в параметрах будут числа $n, k, q$. Чтобы не пояснять их каждый раз, напишу сразу:
* $n, k$ - параметры линейного $(n, k)$ кода
* q - параметр поля $\mathbf{F}_q$

### Задание 1. Генерация порождающих матриц

Будем генерировать сразу канонические матрицы вида $G=[I_k | P]$. Матрица $G$ имеет размеры $k \times n$

Если единичная матрица имеет размеры $k \times k$, то матрица $P$: $k \times n - k$

Для удобства создадим функцию `create_random_matrix`, которая будет генерировать матрицу с заданными размерами над указанным полем и, что самое главное, проверять что строки полученной матрицы ненулевые и линейно независимы. 

Немножко линала: все строки матрицы линейно независимы, когда ранг матрицы совпадает с числом ее строк или столбцов (минимальное из них). В данном случае строк всегда $\leqslant$ чем столбцов. То есть ранг матрицы G должен быть равен k

In [153]:
def create_random_matrix(size: tuple[int, ...], q: int) -> np.ndarray:
    matrix = np.random.randint(q, size=size)
    return matrix

Сама функция генерации порождающей матрицы простая - она просто склеивает единичную матрицу и матрицу $P$

In [154]:
def create_G_matrix(n: int, k: int, q: int) -> np.ndarray:
    I = np.eye(k)
    P = create_random_matrix((k, n - k), q)
    G_matrix = np.hstack((I, P)).astype(int)

    while np.linalg.matrix_rank(G_matrix) != k:  # проверяем на линейную независимость строк
        P = create_random_matrix((k, n - k), q)
        G_matrix = np.hstack((I, P)).astype(int)

    return G_matrix

Наконец, сделаем матрицы с параметрами по заданию:

In [155]:
n1, k1, q1 = 4, 2, 2
n2, k2, q2 = 15, 11, 2
n3, k3, q3 = 6, 4, 3
n4, k4, q4 = 8, 4, 3

G1 = create_G_matrix(n1, k1, q1)
G2 = create_G_matrix(n2, k2, q2)
G3 = create_G_matrix(n3, k3, q3)
G4 = create_G_matrix(n4, k4, q4)

print(f'G1:\n{G1}', 
      f'G2:\n{G2}', 
      f'G3:\n{G3}', 
      f'G4:\n{G4}', sep='\n\n')

G1:
[[1 0 0 1]
 [0 1 1 1]]

G2:
[[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1]]

G3:
[[1 0 0 0 0 2]
 [0 1 0 0 1 1]
 [0 0 1 0 1 0]
 [0 0 0 1 2 1]]

G4:
[[1 0 0 0 1 1 2 2]
 [0 1 0 0 1 0 2 0]
 [0 0 1 0 2 0 2 0]
 [0 0 0 1 0 2 1 1]]


### Задание 2. Получение проверочных матриц

Проверочные матрицы можно получить как $H = [-P^T | I_{n - k}]$. Размеры матрицы $H$: $n - k \times n$

In [156]:
def create_H_matrix(G: np.ndarray, q: int) -> np.ndarray:
    k, n = G.shape
    P = G[:, k:]  # получаем правый кусок матрицы G начиная с индекса k (что и есть матрица P)
    minus_P_T = -P.transpose() % q # все значения в отрицательной транспонированной матрице берем по модулю параметра поля q
    I = np.eye(n - k)
    return np.hstack((minus_P_T, I)).astype(int)

Проверочные матрицы для порождающих из прошлого пункта:

In [157]:
H1 = create_H_matrix(G1, q1)
H2 = create_H_matrix(G2, q2)
H3 = create_H_matrix(G3, q3)
H4 = create_H_matrix(G4, q4)

print(f'H1:\n{H1}', 
      f'H2:\n{H2}', 
      f'H3:\n{H3}', 
      f'H4:\n{H4}', sep='\n\n')

H1:
[[0 1 1 0]
 [1 1 0 1]]

H2:
[[0 1 0 1 0 1 1 0 1 0 1 1 0 0 0]
 [1 0 1 0 1 0 1 1 0 0 1 0 1 0 0]
 [0 0 1 1 0 1 0 1 0 1 0 0 0 1 0]
 [0 1 0 1 1 1 0 0 1 1 1 0 0 0 1]]

H3:
[[0 2 2 1 1 0]
 [1 2 0 2 0 1]]

H4:
[[2 2 1 0 1 0 0 0]
 [2 0 0 1 0 1 0 0]
 [1 1 1 2 0 0 1 0]
 [1 0 0 2 0 0 0 1]]


Для проверки посмотрим, что попарное произведение всех порождающих и транспонированных проверочных матриц равно нулевым матрицам

In [158]:
assert ((G1 @ H1.transpose()) % q1).any() == False
assert ((G2 @ H2.transpose()) % q2).any() == False
assert ((G3 @ H3.transpose()) % q3).any() == False
assert ((G4 @ H4.transpose()) % q4).any() == False

### Задание 3. Декодирование с помощью стандартного расположения

In [159]:
class CodeWord:
    def __init__(self, _arr: list, _q: int):
        self.arr = np.array(_arr)
        self.q = _q
    
    def __add__(self, other: object):
        assert self.q == other.q, 'Different bases'
        return CodeWord((self.arr + other.arr) % self.q, other.q)
    
    def __sub__(self, other: object):
        assert self.q == other.q, 'Different bases'
        return CodeWord((self.arr - other.arr) % self.q, other.q)
    
    def __eq__(self, other: object) -> bool:
        return self.q == other.q and np.array_equal(self.arr, other.arr)

    def __repr__(self) -> str:
        return self.arr.__str__()
    
    __str__ = __repr__
    
    def __hash__(self) -> int:  # чтобы можно было использовать в хэш-таблицах
        return int(np.polyval(self.arr, self.q))
    
    get_id = __hash__

Сначала создадим функцию по генерации кодового множества $C$ по порождающей матрице(все различные линейные комбинации строк)

In [160]:
def generate_code_set(G: np.ndarray, q: int) -> np.ndarray[CodeWord]:
    k, n = G.shape
    c = []
    for j in range(1, q ** k):
        coeffs = np.array([j // (q ** i) % q for i in range(k - 1, -1, -1)])
        c.append(CodeWord(np.dot(coeffs, G) % q, q))

    return np.array(c)

In [161]:
c1 = generate_code_set(G1, q1)
c2 = generate_code_set(G2, q2)
c3 = generate_code_set(G3, q3)
c4 = generate_code_set(G4, q4)

Сначала создадим таблицу стандартного расположения. Для этого для всех возможных кодовых слов посчитаем их индекс и вес. Индекс слова - значение многочлена с основанием $q$ и коэффициентами - самими словами. Так однозначно можем получить индекс из слова и наоборот. Индексы нужны для упрощенного хранения слов (в `array` будут храниться и сравниваться при полученнии лидера именно они).  

Такое хранилище (словарь) делается в функции `__create_id_dict`. Все возможные слова - массивы фиксированной длины с элементами от 0 до $q-1$ (алгоритм по его созданию украл со Stackoverlow и адаптировал, ссылочку уже потерял). Также создадим словарь `used`, где значения - булевые массивы (обозначают было ли использовано ранее данное слово). По мере заполнения таблицы будет менять соответствующие значения в `used`

К сожалению питон не умеет в ordered set, поэтому буду хранить 2 стандартных расположения - одно для быстрой работы (на множествах), второй - для наглядности (на массивах - сохраняется порядок)

In [162]:
class StandardArray:
    def __init__(self, G: np.ndarray, q: int):
        self.__G = G
        self.k, self.n = G.shape
        self.q = q
        self.codeset = generate_code_set(self.__G, q)
        self.array, self.array_printable = self.form()
    

    def __create_id_dict(self) -> tuple[dict, dict]:
        dct = defaultdict(list)
        for i in range(self.q ** self.n):
            curr_id = 0
            curr_weight = 0

            for j in range(self.n):
                i, tmp = divmod(i, self.q)
                curr_weight += tmp
                curr_id += tmp * (self.q ** (self.n - j - 1))
                
            dct[curr_weight].append(curr_id)

        used = dict()
        for k, v in dct.items():
            used[k] = [False] * len(v)
        return dct, used


    def __id_to_word(self, id: int) -> CodeWord:
        a = [0] * self.n
        for i in range(self.n):
            x, y = divmod(id, self.q ** (self.n - i - 1))
            a[i], id = x, y
        return CodeWord(np.array(a), self.q)
    
    @staticmethod
    def mark_used(w: CodeWord, dct: dict, used: dict) -> None:
        weight = int(np.sum(w.arr))
        ind = dct[weight].index(w.get_id())
        used[weight][ind] = True


    def form(self) -> tuple[dict[CodeWord, set[CodeWord]], dict[CodeWord, list[CodeWord]]]:
        array, array_printable = dict(), dict()
        dct, used = self.__create_id_dict()
        minw, curr_ind = 0, 0

        for _ in range(self.q ** (self.n - self.k)):
            while all(used[minw]):
                minw += 1

            curr_ind = used[minw].index(False)
            used[minw][curr_ind] = True
            leader = self.__id_to_word(dct[minw][curr_ind])
            array[leader] = set()
            array_printable[leader] = []

            for w in self.codeset:
                new_w = w + leader
                self.mark_used(new_w, dct, used)
                array[leader].add(new_w.get_id())
                array_printable[leader].append(new_w.get_id())
                
        for item in used.values(): # проверка что все слова использованы
            assert all(item)

        return array, array_printable


    def get_leader(self, w: CodeWord) -> CodeWord:
        w_id = w.get_id()
        for lead, v in self.array.items():
            if w_id in v:
                return lead
        return w

    def decode(self, word: CodeWord) -> CodeWord:
        return word - self.get_leader(word)

    def printarray(self) -> None:
        print('Leader' + "\t" * (self.n // 4 + 1) + 'Words')
        
        for k, v in self.array_printable.items():
            print(k, '|', *[self.__id_to_word(i) for i in v])

Декодирование стандартным расположением реализовано. Выведем таблицу для 1 и 3 кода (для 2 и 4 она не влезает в output). Вы можете конечно проверить что все кодовые слова уникальные, но не советую. Дебажил код очень долго, вроде все работает

In [163]:
sa1 = StandardArray(G1, q1)
sa2 = StandardArray(G2, q2)
sa3 = StandardArray(G3, q3)
sa4 = StandardArray(G4, q4)

In [164]:
sa1.printarray()
sa3.printarray()

Leader		Words
[0 0 0 0] | [0 1 1 1] [1 0 0 1] [1 1 1 0]
[1 0 0 0] | [1 1 1 1] [0 0 0 1] [0 1 1 0]
[0 1 0 0] | [0 0 1 1] [1 1 0 1] [1 0 1 0]
[0 0 1 0] | [0 1 0 1] [1 0 1 1] [1 1 0 0]
Leader		Words
[0 0 0 0 0 0] | [0 0 0 1 2 1] [0 0 0 2 1 2] [0 0 1 0 1 0] [0 0 1 1 0 1] [0 0 1 2 2 2] [0 0 2 0 2 0] [0 0 2 1 1 1] [0 0 2 2 0 2] [0 1 0 0 1 1] [0 1 0 1 0 2] [0 1 0 2 2 0] [0 1 1 0 2 1] [0 1 1 1 1 2] [0 1 1 2 0 0] [0 1 2 0 0 1] [0 1 2 1 2 2] [0 1 2 2 1 0] [0 2 0 0 2 2] [0 2 0 1 1 0] [0 2 0 2 0 1] [0 2 1 0 0 2] [0 2 1 1 2 0] [0 2 1 2 1 1] [0 2 2 0 1 2] [0 2 2 1 0 0] [0 2 2 2 2 1] [1 0 0 0 0 2] [1 0 0 1 2 0] [1 0 0 2 1 1] [1 0 1 0 1 2] [1 0 1 1 0 0] [1 0 1 2 2 1] [1 0 2 0 2 2] [1 0 2 1 1 0] [1 0 2 2 0 1] [1 1 0 0 1 0] [1 1 0 1 0 1] [1 1 0 2 2 2] [1 1 1 0 2 0] [1 1 1 1 1 1] [1 1 1 2 0 2] [1 1 2 0 0 0] [1 1 2 1 2 1] [1 1 2 2 1 2] [1 2 0 0 2 1] [1 2 0 1 1 2] [1 2 0 2 0 0] [1 2 1 0 0 1] [1 2 1 1 2 2] [1 2 1 2 1 0] [1 2 2 0 1 1] [1 2 2 1 0 2] [1 2 2 2 2 0] [2 0 0 0 0 1] [2 0 0 1 2 2] [2 0 0 2 1 0] [2 0

### Задание 4. Декодирование по синдрому

$S_i = e_i * H^\tau$. Получим все векторы ошибок из прошлого задания, помножим каждый на соответствующую транспонированную матрицу и получим синдромы

In [165]:
e1 = list(sa1.array.keys())
e2 = list(sa2.array.keys())
e3 = list(sa3.array.keys())
e4 = list(sa4.array.keys())

In [166]:
class SyndromeDecoder:
    def __init__(self, errors: list[CodeWord], H: np.ndarray, q: int):
        self.H = H
        self.q = q
        self.table = self.create_table(errors)

    def create_table(self, errors: list[CodeWord]) -> dict[tuple, CodeWord]:
        return {tuple(err.arr @ self.H.T % self.q): err for err in errors}

    def decode(self, w: CodeWord) -> CodeWord:
        syn = tuple(w.arr @ self.H.T % self.q)
        err = self.table[syn]
        return w - err
    
    def print_table(self):
        print('Syndrome | Error')
        for syn, err in self.table.items():
            print(''.join(map(str, syn)), '|', err)

Посмотрим на несколько таких таблиц

In [167]:
sd1 = SyndromeDecoder(e1, H1, q1)
sd2 = SyndromeDecoder(e2, H2, q2)
sd3 = SyndromeDecoder(e3, H3, q3)
sd4 = SyndromeDecoder(e4, H4, q4)

sd1.print_table()
print()
sd2.print_table()

Syndrome | Error
00 | [0 0 0 0]
01 | [1 0 0 0]
11 | [0 1 0 0]
10 | [0 0 1 0]

Syndrome | Error
0000 | [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0100 | [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1001 | [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
0110 | [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
1011 | [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
0101 | [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
1100 | [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
0011 | [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
1101 | [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
1000 | [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
0010 | [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
0001 | [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
1111 | [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
1110 | [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
1010 | [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
0111 | [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0]


### Задание 5. Эксперименты

Сначала проверим, что обоими способами декодирование происходит одинаково

In [168]:
for i in range(7):
    w1 = CodeWord(create_random_matrix((n1,), q1), q1)
    w2 = CodeWord(create_random_matrix((n2,), q2), q2)
    w3 = CodeWord(create_random_matrix((n3,), q3), q3)
    w4 = CodeWord(create_random_matrix((n4,), q4), q4)

    print(w1, sa1.decode(w1), sd1.decode(w1), sep='\n')
    print()
    print(w2, sa2.decode(w2), sd2.decode(w2), sep='\n')
    print()
    print(w3, sa3.decode(w3), sd3.decode(w3), sep='\n')
    print()
    print(w4, sa4.decode(w4), sd4.decode(w4), sep='\n')
    print()

[1 1 1 1]
[0 1 1 1]
[0 1 1 1]

[0 1 1 1 0 0 1 0 0 0 1 1 1 0 0]
[0 0 1 1 0 0 1 0 0 0 1 1 1 0 0]
[0 0 1 1 0 0 1 0 0 0 1 1 1 0 0]

[2 0 0 0 2 2]
[1 0 2 0 2 2]
[1 0 2 0 2 2]

[0 1 1 0 2 1 2 2]
[0 1 2 2 2 1 2 2]
[0 1 2 2 2 1 2 2]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 1 0 1 0 0 0 0 0 0 0 1 1 1]
[1 0 1 0 1 0 0 0 0 0 0 0 1 1 1]
[1 0 1 0 1 0 0 0 0 0 0 0 1 1 1]

[0 2 2 0 2 0]
[0 0 2 0 2 0]
[0 0 2 0 2 0]

[0 0 1 1 1 1 1 2]
[2 0 1 1 1 1 1 2]
[2 0 1 1 1 1 1 2]

[1 0 1 1]
[1 0 0 1]
[1 0 0 1]

[0 1 1 0 0 1 0 1 0 0 0 1 1 0 0]
[0 1 1 1 1 1 0 1 0 0 0 1 1 0 0]
[0 1 1 1 1 1 0 1 0 0 0 1 1 0 0]

[1 0 0 0 2 1]
[1 2 0 0 2 1]
[1 2 0 0 2 1]

[0 1 1 0 2 0 2 0]
[0 0 1 0 2 0 2 0]
[0 0 1 0 2 0 2 0]

[1 1 1 1]
[0 1 1 1]
[0 1 1 1]

[1 1 0 1 0 0 1 1 1 1 1 1 1 0 1]
[1 1 1 1 0 0 1 1 1 1 1 1 1 0 1]
[1 1 1 1 0 0 1 1 1 1 1 1 1 0 1]

[0 0 0 1 2 1]
[0 0 0 1 2 1]
[0 0 0 1 2 1]

[0 1 1 2 2 1 2 0]
[2 1 1 2 2 0 1 0]
[2 1 1 2 2 0 1 0]

[1 1 0 1]
[1 0 0 1]
[1 0 0 1]

[0 0 0 1 0 1 1 0 0 1 1 0 0 0 1]
[0 0 0 1 0 1 1 0 0 0 1 0 0 0 1]
[

Как мы видим, декодирование во всех случаях одинаково. Теперь замерим скорость работы

In [169]:
w1s = [CodeWord(create_random_matrix((n1,), q1), q1)] * 1000
w2s = [CodeWord(create_random_matrix((n2,), q2), q2)] * 1000
w3s = [CodeWord(create_random_matrix((n3,), q3), q3)] * 1000
w4s = [CodeWord(create_random_matrix((n4,), q4), q4)] * 1000

In [170]:
%%timeit -n 5
# Standard array
for w1, w2, w3, w4 in zip(w1s, w2s, w3s, w4s):
    sa1.decode(w1)
    sa2.decode(w2)
    sa3.decode(w3)
    sa4.decode(w4)

79.9 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [171]:
%%timeit -n 5
for w1, w2, w3, w4 in zip(w1s, w2s, w3s, w4s):
    sd1.decode(w1)
    sd2.decode(w2)
    sd3.decode(w3)
    sd4.decode(w4)

30.7 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


Как можно заметить, синдромное декодирование получилось в 2 раза быстрее стандартного расположения, чзх? Скорее всего тут проблема в том, что мои кривые руки не умеют писать нормальный оптимизированный код. Должно быть наоборот.

В самом деле, при декодировании стандартным расположением вектор ошибки находится за псевдоконстанту - в моей реализации за $O(q^{n-k})$ и из-за этого она и получилась медленнее. На деле же каждому вектору можно поставить в соответствие синдром, тогда сложность будет $O(1)$. Однако это требует затрат по памяти $O(q^n)$. 

При синдромном же декодировании вектор ошибки получается за $O((n - k) * n)$ (просто умножение матриц), при этом затраты по памяти - $O(q^{n-k})$. 

Скорость будет зависеть и от параметров кода. При некоторых удачных параметрах декодирование по стандартному расположению будет даже быстрее

Что касается исправления ошибок, линейный код с минимальным кодовым расстоянием $d$ может исправить $t = \lfloor \frac{d - 1}{2} \rfloor$ ошибок. Для исправления хотя бы одной нужно d = 3. Посмотрим на минимальные расстояния для наших кодов

In [172]:
def d_min(c: np.ndarray) -> int:
    n = len(c[0].arr)
    d_min = n
    for i in range(len(c)):
        for j in range(i + 1, len(c)):
            d = np.count_nonzero(np.not_equal(c[i].arr, c[j].arr))
            d_min = min(d_min, d)
    return d_min

In [173]:
print(d_min(c1))
print(d_min(c2))
print(d_min(c3))
print(d_min(c4))

2
2
2
3


Тут все зависит от удачи. При данных случайных матрицах получилось, что ошибку может исправить только последний код. Однако на деле $d_{min} \leqslant n$, поэтому такие коды могут исправлять гораздо большее число ошибок 